In [1]:
import numpy as np
import sys
import io
import pandas 
import plotly.express as px
import math 


In [2]:
fin = open('Independent_File.in.txt', mode='r', encoding='utf-8-sig')

In [3]:
#read all the inputs in variables
k1 = float(fin.readline()) #gain #1
k2 = float(fin.readline()) #gain #2
k3 = float(fin.readline()) #gain #3
k4 = float(fin.readline()) #gain #4 
x1 = float(fin.readline()) #initial vehicle position
x2 = float(fin.readline()) #initial vehicle velocity
x3 = float(fin.readline()) #initial wheel angular velocity
x2cmd = float(fin.readline())#command controller of vehicle velocity
tn = int(fin.readline()) #number of time steps
ts = float(fin.readline()) #time steps duration
controlTorque = fin.readline()#names what control torque is currently being used

In [4]:
#define all the variables
A21 = 29.6
A22 = 0.0000559
A31 = 1459.327
A32 = 0.00157
d3 = -14.3201
B = 0.0225
rw = 15.37/12.0
lpk = 0.17
x3cmd =0
if(controlTorque == 'Lyapunov Control Torque'):
    x3cmd = x2cmd/(rw*(1-lpk)) 
else:
    x3cmd = x2cmd/(rw*(1-lpk))

x3Old =x3 #x3 one time step before

l=0 #temporary lambda value

if(x2==0 and x3==0):
    l=0
elif (x2<= rw*x3):
    l = 1-(x2/(rw*x3))
elif(x2>rw*x3):
    l = ((rw*x3)/x2)-1
al = (2*(l/lpk))/(1+((l/lpk)**2)) #attenuation from wheel slip

Tcon =0.0 #temporary control torque value
#if controlTorque=="Sliding Mode Control Torque":
    #Tcon = (1/B)*(((A31-k4*A21)*al) + ((A32+k4*A22)*(x2**2)) +d3-((k3*k4)*(x2-x2cmd))-((k3)*(x3-x3cmd)))#for Sliding Mode Control Torque
#else:
    #Tcon = (1/B)*(((A31-A21)*al) + ((A32+A22)*(x2**2)) +d3-((k3)*(x2-x2cmd)) + ((1/(tn*ts))*(x3-x3Old)))#for Lyapunov Control Torque
    







In [5]:
#Performing the Runge-Kutta Method on Autonomous Vehicle Equations XD

def f(x):
    if(x[1][0]==0 and x[2][0]==0):
        l=0
        print("yuh")
    elif (x[1][0]<= rw*x[2][0]):
        l = 1-(x[1][0]/(rw*x[2][0]))
        print("yuh1")
    elif(x[1][0]>rw*x[2][0]):
        l = ((rw*x[2][0])/x[1][0])-1
        print("yuh2")
    else:
        l=0
        print("yuh3")
        
    al = (2*(l/lpk))/(1+((l/lpk)**2))
    
    x1dot = x[1][0]
    x2dot = A21*al -(A22)*(x[1][0]**2)
    x3dot = (-A31*al)-(A32*(x[1][0]**2))+d3+B*Tcon
    
    return np.array([[x1dot],[x2dot],[x3dot]])




x0= [[x1],[x2],[x3]]
x_values= []
y_values=[]
z_values=[]
Tcon_values=[]
time= []
x2Error = []
x3kError = []
x2MPH = []
x3MPH = []

for i in range(1,(tn+1)):
    
    
    K1 = f(x0)
    K2 = f(x0+np.dot((K1),(ts)*(0.5)))
    K3 = f(x0 + np.dot((K2),(ts)*(0.5)))
    K4 = f(x0+ np.dot(K3,ts))
    xK = x0 + np.dot(np.array(K1) + np.array(np.dot(K2,2)) + np.array(np.dot(K3,2)) + np.array(K4), (ts)*(1/6))
    x3_k1 = x0[2][0]
    x3_k=xK[2][0]
    if(xK[1][0]==0 and xK[2][0]==0):
        l=0
    elif (xK[1][0]<= rw*xK[2][0]):
        l = 1-(xK[1][0]/(rw*xK[2][0]))
    elif(xK[1][0]>rw*xK[2][0]):
        l = ((rw*xK[2][0])/xK[1][0])-1
    alOLD = al
    al = (2*(l/lpk))/(1+((l/lpk)**2))

    x2_k1= x0[1][0]
    #Tcon = (1/B)*(((A31-A21)*al) + ((A32+A22)*(xK[1][0]**2)) +d3-((k3)*(xK[1][0]-x2cmd)) + ((1/(ts))*(x3_k-x3_k1)))

    
    y1=3600/5280*xK[1][0]
    y2=(60/(2*math.pi))*xK[2][0]
    x0=xK 
    if(controlTorque == 'Lyapunov Control Torque'):
        Tcon = Tcon + (1/B)((A31*(al-alOLD) + A32 * ((x0[1][0]**2)-(x2_k1**2)) - k3*(x3_k-x3_k1) + (1/(k2(x3_k-x3_k1)))*(-k4*((x0[1][0]-x2cmd)**2)-k1*(x0[1][0]-x2cmd)*(A21*al-A22*(x0[1][0]**2)))))
    else:
        Tcon = (1.0/B)*(((A31-k4*A21)*al) + ((A32+k4*A22)*(x2**2)) +d3-((k3*k4)*(x2-x2cmd))-((k3)*(x3-x3cmd)))#for Sliding Mode Control Torque
    
    print(i*ts)
    print(x0)
    print(Tcon)
    x_values.append(x0[0][0])
    y_values.append(x0[1][0])
    z_values.append(x0[2][0])
    Tcon_values.append(Tcon)
    x2Error.append(x2-x2cmd)
    x3kError.append(x3_k-x3_k1)
    time.append(i*ts)
    x2MPH.append(y1)
    x3MPH.append(y2)
    

    print()
x_ValuesNum=np.array(x_values)
y_ValuesNum=np.array(y_values)
z_ValuesNum = np.array(z_values)
Tcon_ValuesNum=np.array(Tcon_values)
x2Error_ValuesNum = np.array(x2Error)
x3kError_ValuesNum = np.array(x3kError)
x2MPH_ValuesNum = np.array(x2MPH)
x3MPH_ValuesNum = np.array(x3MPH)

                



yuh
yuh2
yuh3
yuh3
1.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
2.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
3.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
4.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
5.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
6.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
7.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
8.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
9.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
10.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
11.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
12.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
13.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
14.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3
yuh3
yuh3
yuh3
15.0
[[nan]
 [nan]
 [nan]]
261174658907.31714

yuh3


/var/folders/qb/rwbr2swx5mj2yxj7djs76m4w0000gn/T/ipykernel_45260/2527656039.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  l = ((rw*x[2][0])/x[1][0])-1
/var/folders/qb/rwbr2swx5mj2yxj7djs76m4w0000gn/T/ipykernel_45260/2527656039.py:17: RuntimeWarning: invalid value encountered in scalar divide
  al = (2*(l/lpk))/(1+((l/lpk)**2))


In [6]:
def plot_xy_values(x_values, y_values,title):
    # Create a scatter plot using Plotly
    fig = px.line(x=x_values, y=y_values, title = title)

    # Show the plot
    fig.show()

In [7]:
plot_xy_values(time, x_ValuesNum,'Position (ft)v Time Graph') #x1 v time
plot_xy_values(time, y_ValuesNum,'Velocity (ft/s) v Time Graph') #x2 v time
plot_xy_values(time, z_ValuesNum, 'Angular Velocity (rad/sec)v Time Graph') #x3 v time
plot_xy_values(time, Tcon_ValuesNum, 'Lyapunov Control Torque (ft-lbs)v Time Graph') #Torque control v time
plot_xy_values(time, x2Error_ValuesNum, 'Error in Velocity (ft/s) v Time Graph') #error in velocity v time
plot_xy_values(time, x3kError_ValuesNum, 'Angular Velocity Delta (rad/s)v Time Graph') #error in acceleration v time
plot_xy_values(time, x2MPH_ValuesNum, 'Velocity in MPH v Time Graph')
plot_xy_values(time, x3MPH_ValuesNum, 'Acceleration in RPM Graph')
